In [15]:
from IPython.display import IFrame, display
filepath = "../Slides/2022_DistributedComputing_Week7.pdf"
IFrame(src=filepath, width=300, height=200,)

In [1]:
from pyspark import SparkContext
import pyspark
import time

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/01 15:07:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/01 15:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/01 15:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/01 15:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/12/01 15:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/12/01 15:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [2]:
business_sf = sc.textFile(
    "../data/SF_business/filtered_registered_business_sf.tsv")
supervisor_sf = sc.textFile("../data/SF_business/supervisor_sf.tsv")

In [3]:
partition_ct = 4

## 1. Create an RDD which has partition_ct as the number of partitions  in a format of (zip, supervisor_id) order by zip.

In [4]:
sorted_zip_supervisor_id = supervisor_sf.map(lambda x : x.split('\t'))\
                                        .map(lambda x : (int(x[0]), int(x[1])))\
                                        .sortByKey(numPartitions=partition_ct)

## how the data are organized?
- Do same keys locate in the same partition?

In [5]:
sorted_zip_supervisor_id.glom().collect()

[[(94102, 8),
  (94102, 6),
  (94102, 3),
  (94102, 5),
  (94103, 8),
  (94103, 9),
  (94103, 10),
  (94103, 6),
  (94103, 3),
  (94103, 5),
  (94104, 6),
  (94104, 3),
  (94105, 6),
  (94105, 3),
  (94107, 10),
  (94107, 6),
  (94108, 6),
  (94108, 3),
  (94109, 2),
  (94109, 6),
  (94109, 3),
  (94109, 5)],
 [(94110, 8),
  (94110, 11),
  (94110, 9),
  (94110, 10),
  (94111, 6),
  (94111, 3),
  (94112, 7),
  (94112, 8),
  (94112, 11),
  (94112, 9),
  (94112, 10),
  (94114, 7),
  (94114, 8),
  (94114, 5),
  (94115, 2),
  (94115, 1),
  (94115, 5)],
 [(94116, 7),
  (94116, 4),
  (94117, 1),
  (94117, 7),
  (94117, 8),
  (94117, 5),
  (94118, 2),
  (94118, 1),
  (94118, 5),
  (94121, 2),
  (94121, 1),
  (94122, 1),
  (94122, 7),
  (94122, 5),
  (94122, 4),
  (94123, 2),
  (94124, 9),
  (94124, 10)],
 [(94127, 7),
  (94127, 8),
  (94127, 11),
  (94129, 2),
  (94130, 6),
  (94131, 7),
  (94131, 8),
  (94131, 5),
  (94132, 7),
  (94132, 11),
  (94132, 4),
  (94133, 2),
  (94133, 3),
  (94134

## 2. Create an RDD which has partition_ct as the number of partitions in a format of (zip, [name, street, city, zip]) partitioned by zip (hash).

In [6]:
partitioned_zip_business = business_sf.map(lambda x : x.split('\t'))\
                                      .filter(lambda x: len(x[0]) > 0)\
                                      .map(lambda x : (int(x[0]), x[1:]))\
                                      .partitionBy(numPartitions=partition_ct)

## how the data are organized?
- Do same keys locate in the same partition?

In [7]:
partitioned_zip_business.keys().glom().collect()

[[94124,
  94108,
  94108,
  94108,
  94132,
  94108,
  94108,
  94124,
  94124,
  94108,
  94112,
  94112,
  94124,
  94108,
  94124,
  94132,
  94112,
  94112,
  94104,
  94116,
  94108,
  94104,
  0,
  94116,
  94124,
  94108,
  94108,
  94108,
  94108,
  94104,
  94104,
  94108,
  94124,
  94104,
  94108,
  94104,
  94108,
  94108,
  94108,
  94104,
  94108,
  94104,
  94104,
  94104,
  94108,
  94124,
  94124,
  94108,
  94124,
  94104,
  94132,
  94104,
  94108,
  94124,
  94124,
  94124,
  94124,
  94124,
  94124,
  94104,
  94404,
  94404,
  94124,
  94112,
  94108,
  94108,
  94108,
  94108,
  94124,
  94080,
  94124,
  94124,
  94124,
  94124,
  94124,
  94104,
  94108,
  94124,
  94108,
  94104,
  94124,
  94104,
  94108,
  94104,
  94108,
  94108,
  94108,
  94108,
  94112,
  94112,
  94112,
  94112,
  94124,
  94104,
  94108,
  94108,
  94108,
  94588,
  94116,
  94104,
  94104,
  94104,
  94108,
  94140,
  94108,
  94108,
  94104,
  94104,
  94124,
  94108,
  94108,
  941

## 3. Join sorted_zip_supervisor_id and partitioned_zip_business efficiently for any sorted_zip_supervisor_id. (assume that the joined RDDs will be repeatedly used)

In [8]:
sorted_zip_supervisor_id.cache()
partitioned_zip_business.cache()

MapPartitionsRDD[14] at mapPartitions at PythonRDD.scala:145

In [9]:
start = time.time()
joined_supervisor_business = sorted_zip_supervisor_id.leftOuterJoin(partitioned_zip_business)
joined_supervisor_business.collect()
print(time.time() - start)

3.4312539100646973


# Custom Partitioner

## 1) For the range of supervisor ids, create RDD of (supervisor_id, [business name, street, city, state]) within the particion_ct based on the supervisor_id ranges.
- Ex. if partition_ct is 3, and supervisor ids are [1, 2, 3, 4, 5, 6, 7, 8, 9] => first partition should have pairs with supervisor ids of [1,2,3], second paritition should have pairs with supervisor ids of [4,5,6], third partition should have pairs with supervisor ids of [7,8,9]

In [10]:
supervisor_ids = sorted_zip_supervisor_id.values()\
                                         .distinct()\
                                         .sortBy(lambda x : x)\
                                         .collect()

In [11]:
supervisor_ids

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [12]:
def custom_range_partition(key):
    min_id = supervisor_ids[0]
    max_id = supervisor_ids[-1]
    partition_size = (max_id - min_id + 1)/partition_ct
    return int((key - min_id)/partition_size)

In [13]:
custom_range_partition(6)

1

In [14]:
supervisor_business = joined_supervisor_business.values()
supervisor_business_custom_range = supervisor_business.partitionBy(numPartitions=partition_ct,
                                                                   partitionFunc=custom_range_partition)
supervisor_business_custom_range.saveAsTextFile('supervisor_business_custom_range')

Py4JJavaError: An error occurred while calling o221.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/dwoodbridge/Class/2022_MSDS694/2022-msds694-example/Week7/supervisor_business_custom_range already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRD

## 2)  Store pairs of (supervisor_id, [business name, street, city, state]) within 2 partitions, based on whether the supervisor_id is even or odd.

In [ ]:
def odd_even_partitioner(key):
    return key%2

In [ ]:
supervisor_business_odd_even = supervisor_business.partitionBy(2, odd_even_partitioner)
supervisor_business_odd_even.saveAsTextFile('supervisor_business_odd_even')

In [ ]:
sc.stop()